# Get Network Activations for Test Images
*Written by Viviane Clay*
### Load Dependencies

In [3]:
import os
import numpy as np;
import tensorflow as tf
import tensorflow.contrib.layers as c_layers
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
import scipy

In [4]:
print(tf.__version__)

1.8.0


### Load Test Data
Data set and trained models can be downloaded here: http://dx.doi.org/10.17632/zdh4d5ws2z.2 For the paper a subset of the full test set is used. To do this exclude all folders with [x,x,x,1] which are folders containing images labeled as 'puzzle'.

In [10]:
balancedTestSetPath = 'F:\STUDY\MS Cognitive Science IITD\Internship\RTG Computational Cognition\Code/BalancedTestSetAll'
datagen = ImageDataGenerator()
#removed validation_split=0
test_bal = datagen.flow_from_directory(balancedTestSetPath, class_mode='sparse',
                                       batch_size=1600,shuffle=False,subset="training",target_size=(168,168))
# change batch_size to 1600 if also testing for puzzles
realLabel = []
for c,v in test_bal.class_indices.items():
    c_ext = np.fromstring(c[1:-1], dtype=int, sep=', ')
    realLabel.append(c_ext)

def getRealLabel(labelBatch,RL):
    newLB = []
    for label in labelBatch:
        l = RL[int(label)]
        newLB.append(l)
    return newLB

def getConceptSubset(AllConceptExamples, numExp):
    subset = np.random.randint(0,AllConceptExamples.shape[0],numExp)
    trainExp = AllConceptExamples[subset]
    mask = np.ones(AllConceptExamples.shape, bool)
    mask[subset] = False
    testExp = AllConceptExamples[mask]
    return trainExp,  testExp

Found 1600 images belonging to 17 classes.


In [11]:
obs,label = test_bal.next()
obs = obs/255
y = np.array(getRealLabel(label, realLabel))

In [12]:
y.shape

(1600, 4)

In [13]:
concept_def = [y[:,0]==1, y[:,0]==2, y[:,0]==3, y[:,0]==4, y[:,1]==1, y[:,2]==1]#y[:,0]==0, 
np.where(concept_def[5])

(array([ 250,  251,  252,  253,  254,  255,  256,  257,  258,  259,  260,
         261,  262,  263,  264,  265,  266,  267,  268,  269,  270,  271,
         272,  273,  274,  275,  276,  277,  278,  279,  280,  281,  282,
         283,  284,  285,  286,  287,  288,  289,  290,  291,  292,  293,
         294,  295,  296,  297,  298,  299,  650,  651,  652,  653,  654,
         655,  656,  657,  658,  659,  660,  661,  662,  663,  664,  665,
         666,  667,  668,  669,  670,  671,  672,  673,  674,  675,  676,
         677,  678,  679,  680,  681,  682,  683,  684,  685,  686,  687,
         688,  689,  690,  691,  692,  693,  694,  695,  696,  697,  698,
         699,  900,  901,  902,  903,  904,  905,  906,  907,  908,  909,
         910,  911,  912,  913,  914,  915,  916,  917,  918,  919,  920,
         921,  922,  923,  924,  925,  926,  927,  928,  929,  930,  931,
         932,  933,  934,  935,  936,  937,  938,  939,  940,  941,  942,
         943,  944,  945,  946,  947, 

In [14]:
obs.shape

(1600, 168, 168, 3)

In [7]:
vis_obs = np.load('F:\STUDY\MS Cognitive Science IITD\Internship\RTG Computational Cognition\Code\FastConceptMapping-main\data\\test_run\\visobs.npy')

In [8]:
vis_obs.shape

(4000, 168, 168, 3)

In [9]:
vec_obs = np.load('F:\STUDY\MS Cognitive Science IITD\Internship\RTG Computational Cognition\Code\FastConceptMapping-main\data\\test_run\\vecobs.npy')

In [10]:
vec_obs.shape

(1, 4000, 8)

In [11]:
vec_obs = vec_obs.reshape(4000,8)

In [12]:
# next visual and vector obs
next_vis_obs = []
next_vec_obs = []
for i in vis_obs[1:]:
    next_vis_obs.append(i)
for j in vec_obs[1:]:
    next_vec_obs.append(j)

next_vis_obs = np.array(next_vis_obs)
next_vec_obs = np.array(next_vec_obs)

In [14]:
next_vec_obs.shape
next_vis_obs.shape

(3999, 168, 168, 3)

In [16]:
# obs me last vala remove kr dete haain, cuz it's next doesn't exist in next_obs
vis_obs = vis_obs[:-1]
vec_obs = vec_obs[:-1]

In [18]:
vis_obs.shape
vec_obs.shape

(3999, 8)

## Get Network Activations
### Embodied Agent
Adapt paths to model checkpoints. Checkpoints can be found in the folder 'data/agent_checkpoints'

In [14]:
#external rewards agent
ckpt_path_ext = 'F:\STUDY\MS Cognitive Science IITD\Internship\RTG Computational Cognition\Code\FastConceptMapping-main\data\\agent_checkpoints\TowerF4_Baseline-0\LearningBrain/model-30100120.cptk'
#curious & external rewards agent
ckpt_path_extint = 'F:\STUDY\MS Cognitive Science IITD\Internship\RTG Computational Cognition\Code\Models\curious_plus_ext\LearningBrain/model-36351394.cptk'
#curious agent
ckpt_path_int = 'F:\STUDY\MS Cognitive Science IITD\Internship\RTG Computational Cognition\Code\Models\curious_minus_ext/model-82450000.cptk'

tf.reset_default_graph()

def swish(input_activation):
    """Swish activation function. For more info: https://arxiv.org/abs/1710.05941"""
    return tf.multiply(input_activation, tf.nn.sigmoid(input_activation))

def create_global_steps():
    """Creates TF ops to track and increment global training step."""
    global_step = tf.Variable(0, name="global_step", trainable=False, dtype=tf.int32)
    increment_step = tf.assign(global_step, tf.add(global_step, 1))
    return global_step, increment_step


global_step, increment_step = create_global_steps()

o_size_h = 168
o_size_w = 168
vec_obs_size = 8
num_layers = 2
h_size = 256
h_size_vec = 256
            
visual_in = tf.placeholder(shape=[None, o_size_h, o_size_w, 3], dtype=tf.float32,name="visual_observation_0")

running_mean = tf.get_variable("running_mean", [vec_obs_size],trainable=False, dtype=tf.float32,initializer=tf.zeros_initializer())
running_variance = tf.get_variable("running_variance", [vec_obs_size],trainable=False,dtype=tf.float32,initializer=tf.ones_initializer())

def create_vector_observation_encoder(observation_input, h_size, activation, num_layers, scope,reuse):
    with tf.variable_scope(scope):
        hidden_vec = observation_input
        for i in range(num_layers):
            hidden_vec = tf.layers.dense(hidden_vec, h_size, activation=activation, reuse=reuse,name="hidden_{}".format(i),kernel_initializer=c_layers.variance_scaling_initializer(1.0))
    return hidden_vec

def create_visual_observation_encoder(image_input, h_size, activation, num_layers, scope,reuse):
    with tf.variable_scope(scope):
        conv1 = tf.layers.conv2d(image_input, 16, kernel_size=[8, 8], strides=[4, 4],activation=tf.nn.elu, reuse=reuse, name="conv_1")
        conv2 = tf.layers.conv2d(conv1, 32, kernel_size=[4, 4], strides=[2, 2],activation=tf.nn.elu, reuse=reuse, name="conv_2")
        hidden_vis = c_layers.flatten(conv2)

    with tf.variable_scope(scope + '/' + 'flat_encoding'):
        hidden_flat = create_vector_observation_encoder(hidden_vis, h_size, activation,num_layers, scope, reuse)
    return hidden_flat


visual_encoders = []
hidden_state, hidden_visual = None, None

encoded_visual = create_visual_observation_encoder(visual_in,h_size,swish,num_layers,"main_graph_0_encoder0", False)
visual_encoders.append(encoded_visual)
hidden_visual = tf.concat(visual_encoders, axis=1)

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

saver = tf.train.Saver()

saver.restore(sess, ckpt_path_ext)

INFO:tensorflow:Restoring parameters from F:\STUDY\MS Cognitive Science IITD\Internship\RTG Computational Cognition\Code\FastConceptMapping-main\data\agent_checkpoints\TowerF4_Baseline-0\LearningBrain/model-30100120.cptk


### Curiosity Encodings

In [16]:
# curious agent curiosity encodings

ckpt_path_int = 'F:\STUDY\MS Cognitive Science IITD\Internship\RTG Computational Cognition\Code\Models\curious_minus_ext/model-82450000.cptk'

#curious & external rewards agent
ckpt_path_extint = 'F:\STUDY\MS Cognitive Science IITD\Internship\RTG Computational Cognition\Code\Models\curious_plus_ext\LearningBrain/model-36351394.cptk'

tf.reset_default_graph()

def swish(input_activation):
    """Swish activation function. For more info: https://arxiv.org/abs/1710.05941"""
    return tf.multiply(input_activation, tf.nn.sigmoid(input_activation))

o_size_h = 168
o_size_w = 168
curiosity_enc_size = 64

visual_in = tf.placeholder(shape=[None, o_size_h, o_size_w, 3], dtype=tf.float32,name="visual_observation_0")


def create_vector_observation_encoder(observation_input, h_size, activation, num_layers, scope,reuse):
    with tf.variable_scope(scope):
        hidden_vec = observation_input
        for i in range(num_layers):
            hidden_vec = tf.layers.dense(hidden_vec, h_size, activation=activation, reuse=reuse,name="hidden_{}".format(i),kernel_initializer=c_layers.variance_scaling_initializer(1.0))
    return hidden_vec

def create_visual_observation_encoder(image_input, h_size, activation, num_layers, scope,reuse):
    with tf.variable_scope(scope):
        conv1 = tf.layers.conv2d(image_input, 16, kernel_size=[8, 8], strides=[4, 4],activation=tf.nn.elu, reuse=reuse, name="conv_1")
        conv2 = tf.layers.conv2d(conv1, 32, kernel_size=[4, 4], strides=[2, 2],activation=tf.nn.elu, reuse=reuse, name="conv_2")
        hidden_vis = c_layers.flatten(conv2)

    with tf.variable_scope(scope + '/' + 'flat_encoding'):
        hidden_flat = create_vector_observation_encoder(hidden_vis, h_size, activation,num_layers, scope, reuse)
    return hidden_flat

def create_curiosity_encoder(visual_in, curiosity_enc_size):
        """.
        Used for implementation of ﻿Curiosity-driven Exploration by Self-supervised Prediction
        See https://arxiv.org/abs/1705.05363 for more details.
        :return: current state visual encoding
        """
        
        # Create the encoder ops for current visual input. Note that these encoders are siamese.
        encoded_visual = create_visual_observation_encoder(visual_in, curiosity_enc_size,
                                                                swish, 1, "stream_{}_visual_obs_encoder"
                                                                .format(0), False)

            
        return encoded_visual

    

visual_encoders = []
# encoded_state_list = []


encoded_visual = create_curiosity_encoder(visual_in, curiosity_enc_size)
# encoded_state_list = create_curiosity_encoder(visual_in, curiosity_enc_size)



visual_encoders.append(encoded_visual)
hidden_visual = tf.concat(visual_encoders, axis=1)

# encoded_state_list.append(hidden_visual)

# encoded_state_list.append(encoded_vector_obs)


    
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

saver = tf.train.Saver()

saver.restore(sess, ckpt_path_int)
# saver.restore(sess, ckpt_path_extint)


encInt2 = sess.run(hidden_visual, feed_dict={visual_in: obs})
# encExtInt2 = sess.run(hidden_visual, feed_dict={visual_in: obs})

np.save(balancedTestSetPath + '/SortedencInt2.npy',encInt2)
# np.save(balancedTestSetPath + '/SortedencExtInt2.npy',encExtInt2)

INFO:tensorflow:Restoring parameters from F:\STUDY\MS Cognitive Science IITD\Internship\RTG Computational Cognition\Code\Models\curious_minus_ext/model-82450000.cptk


In [17]:
encInt2.shape

(1600, 64)

In [10]:
encBal = sess.run(hidden_visual, feed_dict={visual_in: obs})

In [11]:
np.save(balancedTestSetPath + '/encInt.npy',encBal)

In [15]:
#change last line of network code to saver.restore(sess, ckpt_path_ext)
encExt = sess.run(hidden_visual, feed_dict={visual_in: obs})

In [16]:
np.save(balancedTestSetPath + '/encExt.npy',encExt)

In [7]:
#change last line of network code to saver.restore(sess, ckpt_path_extint)
encExtInt = sess.run(hidden_visual, feed_dict={visual_in: obs})

In [8]:
np.save(balancedTestSetPath + '/encExtInt.npy',encExtInt)

## Model predictions vs Ground truth (in curiosity encodings)
to check where the error comes from

### Saving encoding states

In [79]:
# curious agent curiosity encodings

ckpt_path_int = 'F:\STUDY\MS Cognitive Science IITD\Internship\RTG Computational Cognition\Code\Models\curious_minus_ext/model-82450000.cptk'

#curious & external rewards agent
ckpt_path_extint = 'F:\STUDY\MS Cognitive Science IITD\Internship\RTG Computational Cognition\Code\Models\curious_plus_ext\LearningBrain/model-36351394.cptk'

tf.reset_default_graph()

def swish(input_activation):
    """Swish activation function. For more info: https://arxiv.org/abs/1710.05941"""
    return tf.multiply(input_activation, tf.nn.sigmoid(input_activation))

def create_global_steps():
    """Creates TF ops to track and increment global training step."""
    global_step = tf.Variable(0, name="global_step", trainable=False, dtype=tf.int32)
    increment_step = tf.assign(global_step, tf.add(global_step, 1))
    return global_step, increment_step


global_step, increment_step = create_global_steps()

o_size_h = 168
o_size_w = 168
curiosity_enc_size = 64 #128 for extint
vec_obs_size = 8

visual_in = tf.placeholder(shape=[None, o_size_h, o_size_w, 3], dtype=tf.float32,name="visual_observation_0")
next_visual_in = tf.placeholder(shape=[None, o_size_h, o_size_w, 3], dtype=tf.float32,name="next_visual_observation_0")
vector_in = tf.placeholder(shape=[None, vec_obs_size], dtype=tf.float32, name='vector_observation_0')
next_vector_in = tf.placeholder(shape=[None, vec_obs_size], dtype=tf.float32, name='next_vector_observation_0')

def create_vector_observation_encoder(observation_input, h_size, activation, num_layers, scope,reuse):
    with tf.variable_scope(scope):
        hidden_vec = observation_input
        for i in range(num_layers):
            hidden_vec = tf.layers.dense(hidden_vec, h_size, activation=activation, reuse=reuse,name="hidden_{}".format(i),kernel_initializer=c_layers.variance_scaling_initializer(1.0))
    return hidden_vec

def create_visual_observation_encoder(image_input, h_size, activation, num_layers, scope,reuse):
    with tf.variable_scope(scope):
        conv1 = tf.layers.conv2d(image_input, 16, kernel_size=[8, 8], strides=[4, 4],activation=tf.nn.elu, reuse=reuse, name="conv_1")
        conv2 = tf.layers.conv2d(conv1, 32, kernel_size=[4, 4], strides=[2, 2],activation=tf.nn.elu, reuse=reuse, name="conv_2")
        hidden_vis = c_layers.flatten(conv2)

    with tf.variable_scope(scope + '/' + 'flat_encoding'):
        hidden_flat = create_vector_observation_encoder(hidden_vis, h_size, activation,num_layers, scope, reuse)
    return hidden_flat

def create_curiosity_encoders(visual_in,next_visual_in,vector_in,next_vector_in,curiosity_enc_size):
        """
        Creates state encoders for current and future observations.
        Used for implementation of ﻿Curiosity-driven Exploration by Self-supervised Prediction
        See https://arxiv.org/abs/1705.05363 for more details.
        :return: current and future state encoder tensors.
        """

#         if vis_obs_size > 0:
#             next_visual_in = []
#             visual_encoders = []
#             next_visual_encoders = []
            
            # Create input ops for next (t+1) visual observations.
#             next_visual_input = self.create_visual_input(self.brain.camera_resolutions[i],
#                                                          name="next_visual_observation_" + str(i))
#             next_visual_in.append(next_visual_input)

            # Create the encoder ops for current and next visual input. Not that these encoders are siamese.
        encoded_visual = create_visual_observation_encoder(visual_in, curiosity_enc_size,
                                                                swish, 1, "stream_{}_visual_obs_encoder"
                                                                .format(0), False)

        encoded_next_visual = create_visual_observation_encoder(next_visual_in,
                                                                     curiosity_enc_size,
                                                                     swish, 1,
                                                                     "stream_{}_visual_obs_encoder".format(0),
                                                                     True)
#             visual_encoders.append(encoded_visual)
#             next_visual_encoders.append(encoded_next_visual)

#             hidden_visual = tf.concat(visual_encoders, axis=1)
#             hidden_next_visual = tf.concat(next_visual_encoders, axis=1)
#             encoded_state_list.append(hidden_visual)
#             encoded_next_state_list.append(hidden_next_visual)

#         if vec_obs_size > 0:
            # Create the encoder ops for current and next vector input. Not that these encoders are siamese.
            # Create input op for next (t+1) vector observation.
            

        encoded_vector_obs = create_vector_observation_encoder(vector_in,
                                                                    curiosity_enc_size,
                                                                    swish, 2, "vector_obs_encoder",
                                                                    False)
        encoded_next_vector_obs = create_vector_observation_encoder(next_vector_in,
                                                                         curiosity_enc_size,
                                                                         swish, 2,
                                                                        "vector_obs_encoder",
                                                                         True)
#             encoded_state_list.append(encoded_vector_obs)
#             encoded_next_state_list.append(encoded_next_vector_obs)


#         encoded_state = tf.concat(encoded_state_list, axis=1)
#         self.enc_cur_state = tf.identity(encoded_state, name='enc_cur_state')
#         encoded_next_state = tf.concat(encoded_next_state_list, axis=1)
        return encoded_visual, encoded_next_visual, encoded_vector_obs, encoded_next_vector_obs
    

visual_encoders = []
next_visual_encoders = []
encoded_state_list = []
encoded_next_state_list = []

encoded_visual, encoded_next_visual, encoded_vector_obs, encoded_next_vector_obs = create_curiosity_encoders(visual_in,next_visual_in,vector_in,next_vector_in,curiosity_enc_size)


visual_encoders.append(encoded_visual)
next_visual_encoders.append(encoded_next_visual)

hidden_visual = tf.concat(visual_encoders, axis=1)
hidden_next_visual = tf.concat(next_visual_encoders, axis=1)

encoded_state_list.append(hidden_visual)
encoded_next_state_list.append(hidden_next_visual)

#vector obs
encoded_state_list.append(encoded_vector_obs)
encoded_next_state_list.append(encoded_next_vector_obs)

encoded_state = tf.concat(encoded_state_list, axis=1)
# enc_cur_state = tf.identity(encoded_state, name='enc_cur_state')
encoded_next_state = tf.concat(encoded_next_state_list, axis=1)

    
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

saver = tf.train.Saver()

# saver.restore(sess, ckpt_path_extint)
saver.restore(sess, ckpt_path_int)

# For ext + int
# curiosity_encoding_ExtInt = sess.run(encoded_state, feed_dict={visual_in: vis_obs, next_visual_in: next_vis_obs, 
#                                                                  vector_in: vec_obs, next_vector_in: next_vec_obs})

# curiosity_encoding_nextstate_ExtInt = sess.run(encoded_next_state, feed_dict={visual_in: vis_obs, next_visual_in: next_vis_obs, 
#                                                                  vector_in: vec_obs, next_vector_in: next_vec_obs})

# For int
curiosity_encoding_Int = sess.run(encoded_state, feed_dict={visual_in: vis_obs, next_visual_in: next_vis_obs, 
                                                                 vector_in: vec_obs, next_vector_in: next_vec_obs})

curiosity_encoding_nextstate_Int = sess.run(encoded_next_state, feed_dict={visual_in: vis_obs, next_visual_in: next_vis_obs, 
                                                                 vector_in: vec_obs, next_vector_in: next_vec_obs})


np.save('curiosity_encoding_Int.npy',curiosity_encoding_Int)
np.save('curiosity_encoding_nextstate_Int.npy',curiosity_encoding_nextstate_Int)

# np.save('curiosity_encoding_ExtInt.npy',curiosity_encoding_ExtInt)
# np.save('curiosity_encoding_nextstate_ExtInt.npy',curiosity_encoding_nextstate_ExtInt)

INFO:tensorflow:Restoring parameters from F:\STUDY\MS Cognitive Science IITD\Internship\RTG Computational Cognition\Code\Models\curious_minus_ext/model-82450000.cptk


In [77]:
np.array(curiosity_encoding_nextstate_ExtInt).shape

(3999, 256)

In [78]:
curiosity_encoding_ExtInt[-1]

array([-0.00367654, -0.27708212, -0.24987513, -0.01284845, -0.2676933 ,
       -0.27519462, -0.23386034, -0.00631044, -0.2529201 , -0.2630602 ,
       -0.26540604, -0.2591467 , -0.25666052, -0.26759818, -0.27160275,
       -0.2776537 , -0.2306562 , -0.22052696, -0.2683818 , -0.01353922,
       -0.26763335, -0.03174633, -0.27825394, -0.03178577, -0.27435848,
       -0.02467726, -0.24070746, -0.18899806, -0.0331885 , -0.2570049 ,
       -0.01087864, -0.27800736, -0.27582473, -0.21213533, -0.01590122,
       -0.26669854, -0.01725548, -0.02650894, -0.21572542, -0.24032596,
       -0.24241728, -0.2744831 , -0.27536508, -0.22348848, -0.27369907,
       -0.01261332, -0.26492476, -0.27843088, -0.25659516, -0.27824628,
       -0.1133415 , -0.2577189 , -0.03740476, -0.2650148 , -0.27785304,
       -0.24562898, -0.21821555, -0.2739261 , -0.25572857, -0.27845117,
       -0.2777181 , -0.25902605, -0.26506665, -0.24935463, -0.26281443,
       -0.26471642, -0.27762637, -0.27742586, -0.25887793, -0.00

In [80]:
curiosity_encoding_nextstate_Int[-1]

array([-1.9395639e-01, -2.1538323e-01, -2.4449641e-01, -2.7351409e-01,
       -2.5941759e-01, -2.7187482e-01, -2.7513552e-01, -1.0053566e-01,
       -2.2856171e-01, -2.7323657e-01, -3.7801869e-02, -2.6427484e-01,
       -2.6737252e-01, -2.5008720e-01, -6.1797039e-03, -6.7955725e-02,
       -1.4259106e-01, -2.7057731e-01, -2.6583448e-01, -2.6172015e-01,
       -1.6946465e-01, -3.1065980e-02, -2.3008010e-01, -2.7550307e-01,
       -2.7095786e-01, -2.6026446e-01, -2.3612602e-01, -2.4449435e-01,
       -2.5509989e-01, -1.3877296e-02, -2.5493723e-01, -2.3772399e-01,
       -2.6994812e-01, -2.7833155e-01, -7.8720979e-02, -2.7699345e-01,
       -1.6600909e-03, -2.6169819e-01, -2.7794048e-01, -2.7799729e-01,
       -2.2737677e-01, -2.0436624e-01, -2.6688579e-02, -2.1890354e-01,
       -2.7790481e-01, -7.3809922e-02, -1.7890589e-01, -2.7844119e-01,
       -2.7421746e-01, -2.1711354e-01, -2.7698117e-01, -2.0790361e-01,
       -3.3025265e-02, -1.4478830e-01, -1.4179970e-01, -2.7617061e-01,
      

## Predictions and Forward/Backward Losses

In [53]:
actions = np.load('F:\STUDY\MS Cognitive Science IITD\Internship\RTG Computational Cognition\Code\FastConceptMapping-main\data\\actions.npy')

In [54]:
actions = np.array(actions[:-1],dtype=int)
actions.shape

(3999, 1, 4)

In [58]:
actions = actions.reshape(3999,4)

### Forward model

In [98]:
"""FOR INT CASE"""
encoded_state_int = np.load("curiosity_encoding_Int.npy")
encoded_nextstate_int = np.load("curiosity_encoding_nextstate_Int.npy")

# encoded_state_int = np.array(encoded_state_int,dtype=float)
# encoded_state_int = encoded_state_int.reshape(3999,1,128)

tf.reset_default_graph()

def swish(input_activation):
    """Swish activation function. For more info: https://arxiv.org/abs/1710.05941"""
    return tf.multiply(input_activation, tf.nn.sigmoid(input_activation))

def create_global_steps():
    """Creates TF ops to track and increment global training step."""
    global_step = tf.Variable(0, name="global_step", trainable=False, dtype=tf.int32)
    increment_step = tf.assign(global_step, tf.add(global_step, 1))
    return global_step, increment_step


global_step, increment_step = create_global_steps()

o_size_h = 168
o_size_w = 168
curiosity_enc_size = 64 #64 for int
vec_obs_size = 8
curiosity_strength=1.0
vis_obs_size = 1

encoded_state = tf.placeholder(shape=[None, 128], dtype=tf.float32,name="encoded_state")
encoded_next_state = tf.placeholder(shape=[None, 128], dtype=tf.float32,name="encoded_next_state")
sel_action = tf.placeholder(shape=[None, 4], dtype=tf.float32,name="sel_action")


def create_forward_model(encoded_state, encoded_next_state, selected_action):
#         encoded_next_state
        """
        Creates forward model TensorFlow ops for Curiosity module.
        Predicts encoded future state based on encoded current state and given action.
        :param encoded_state: Tensor corresponding to encoded current state.
        :param encoded_next_state: Tensor corresponding to encoded next state.
        :param selected_actions: actions taken
        """
        combined_input = tf.concat([encoded_state, selected_action], axis=1)
        hidden = tf.layers.dense(combined_input, 256, activation=swish)
        # We compare against the concatenation of all observation streams, hence `self.vis_obs_size + int(self.vec_obs_size > 0)`.
        pred_next_state = tf.layers.dense(hidden, curiosity_enc_size * (vis_obs_size + int(vec_obs_size > 0)),
                                          activation=None)
        

        squared_difference = 0.5 * tf.reduce_sum(tf.squared_difference(pred_next_state, encoded_next_state), axis=1)
        intrinsic_reward = tf.clip_by_value(curiosity_strength * squared_difference, 0, 1)
        forward_loss = tf.reduce_mean(tf.dynamic_partition(data=squared_difference,partitions=[0]*3999
                                                           ,num_partitions=2)[1])
        
        return pred_next_state, squared_difference

pred_next_states = []
forward_losses = []

pred_next_state, forward_loss = create_forward_model(encoded_state, encoded_next_state, sel_action)
    

pred_next_states.append(pred_next_state)
forward_losses.append(forward_loss)
    
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

saver = tf.train.Saver()

saver.restore(sess, ckpt_path_int)


# For int
pred_next_states_Int = sess.run(pred_next_states, feed_dict={encoded_state: encoded_state_int, encoded_next_state: encoded_nextstate_int, sel_action: actions})
forward_losses_Int = sess.run(forward_losses, feed_dict={encoded_state: encoded_state_int, encoded_next_state: encoded_nextstate_int, sel_action: actions})



# np.save(balancedTestSetPath + '/curiosity_encoding_Int.npy',curiosity_encoding_Int)

INFO:tensorflow:Restoring parameters from F:\STUDY\MS Cognitive Science IITD\Internship\RTG Computational Cognition\Code\Models\curious_minus_ext/model-82450000.cptk


NotFoundError: Key dense/bias not found in checkpoint
	 [[Node: save/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_INT32], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save/Const_0_0, save/RestoreV2/tensor_names, save/RestoreV2/shape_and_slices)]]

Caused by op 'save/RestoreV2', defined at:
  File "c:\users\varad srivastava\anaconda3\envs\ml-agents\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\varad srivastava\anaconda3\envs\ml-agents\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\varad srivastava\anaconda3\envs\ml-agents\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\users\varad srivastava\anaconda3\envs\ml-agents\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "c:\users\varad srivastava\anaconda3\envs\ml-agents\lib\site-packages\ipykernel\kernelapp.py"

In [90]:
np.save('pred_next_states_Int.npy',pred_next_states_Int)
np.save('forward_losses_Int.npy',forward_losses_Int)

In [97]:
# pred_next_states_Int[0][-1]
np.array(forward_losses_Int[0])
forward_losses_Int[0]

array([2.4832752, 2.6360366, 2.645959 , ..., 2.5831168, 2.4899285,
       2.414308 ], dtype=float32)

In [85]:
"""FOR EXT + INT CASE"""
encoded_state_extint = np.load("curiosity_encoding_ExtInt.npy")
encoded_nextstate_extint = np.load("curiosity_encoding_nextstate_ExtInt.npy")


tf.reset_default_graph()

def swish(input_activation):
    """Swish activation function. For more info: https://arxiv.org/abs/1710.05941"""
    return tf.multiply(input_activation, tf.nn.sigmoid(input_activation))

def create_global_steps():
    """Creates TF ops to track and increment global training step."""
    global_step = tf.Variable(0, name="global_step", trainable=False, dtype=tf.int32)
    increment_step = tf.assign(global_step, tf.add(global_step, 1))
    return global_step, increment_step


global_step, increment_step = create_global_steps()

o_size_h = 168
o_size_w = 168
curiosity_enc_size = 128 #64 for int
vec_obs_size = 8
curiosity_strength=1.0
vis_obs_size = 1

encoded_state = tf.placeholder(shape=[None, 256], dtype=tf.float32,name="encoded_state")
encoded_next_state = tf.placeholder(shape=[None, 256], dtype=tf.float32,name="encoded_next_state")
sel_action = tf.placeholder(shape=[None, 4], dtype=tf.float32,name="sel_action")


def create_forward_model(encoded_state, encoded_next_state, selected_action):
#         encoded_next_state
        """
        Creates forward model TensorFlow ops for Curiosity module.
        Predicts encoded future state based on encoded current state and given action.
        :param encoded_state: Tensor corresponding to encoded current state.
        :param encoded_next_state: Tensor corresponding to encoded next state.
        :param selected_actions: actions taken
        """
        combined_input = tf.concat([encoded_state, selected_action], axis=1)
        hidden = tf.layers.dense(combined_input, 256, activation=swish)
        # We compare against the concatenation of all observation streams, hence `self.vis_obs_size + int(self.vec_obs_size > 0)`.
        pred_next_state = tf.layers.dense(hidden, curiosity_enc_size * (vis_obs_size + int(vec_obs_size > 0)),
                                          activation=None)
        

        squared_difference = 0.5 * tf.reduce_sum(tf.squared_difference(pred_next_state, encoded_next_state), axis=1)
        intrinsic_reward = tf.clip_by_value(curiosity_strength * squared_difference, 0, 1)
        forward_loss = tf.reduce_mean(tf.dynamic_partition(data=squared_difference,partitions=[0]*3999
                                                           ,num_partitions=2)[1])
        
        return pred_next_state, forward_loss

pred_next_states = []
forward_losses = []

pred_next_state, forward_loss = create_forward_model(encoded_state, encoded_next_state, sel_action)
    

pred_next_states.append(pred_next_state)
forward_losses.append(forward_loss)
    
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)


# For extint
pred_next_states_ExtInt = sess.run(pred_next_states, feed_dict={encoded_state: encoded_state_extint, encoded_next_state: encoded_nextstate_extint, sel_action: actions})
forward_losses_ExtInt = sess.run(forward_losses, feed_dict={encoded_state: encoded_state_extint, encoded_next_state: encoded_nextstate_extint, sel_action: actions})



# np.save(balancedTestSetPath + '/curiosity_encoding_Int.npy',curiosity_encoding_Int)

In [87]:
pred_next_states_ExtInt[-1]
# forward_losses_ExtInt

array([[ 0.05881967,  0.05111243,  0.00898644, ..., -0.06855196,
        -0.08051717,  0.10800347],
       [ 0.06480721,  0.04491611,  0.01369487, ..., -0.0662618 ,
        -0.06143788,  0.11779893],
       [ 0.04378022,  0.06110382, -0.01006842, ..., -0.06917566,
        -0.05532652,  0.12691106],
       ...,
       [ 0.10704052,  0.09714859,  0.02385978, ..., -0.01954409,
        -0.08360283,  0.12029169],
       [ 0.06831752,  0.05936614, -0.00956372, ..., -0.07439092,
        -0.07589203,  0.10902264],
       [ 0.05609108,  0.06606102, -0.00768268, ..., -0.0591695 ,
        -0.07701965,  0.12295669]], dtype=float32)

In [88]:
np.save('pred_next_states_ExtInt.npy',pred_next_states_ExtInt)
np.save('forward_losses_ExtInt.npy',forward_losses_ExtInt)

### Inverse Model

In [50]:
"""FOR INT CASE"""
encoded_state_int = np.load("curiosity_encoding_Int.npy")
encoded_nextstate_int = np.load("curiosity_encoding_nextstate_Int.npy")
# encoded_state_int = np.array(encoded_state_int,dtype=float)
# encoded_nextstate_int = np.array(encoded_nextstate_int,dtype=float)

tf.reset_default_graph()

def swish(input_activation):
    """Swish activation function. For more info: https://arxiv.org/abs/1710.05941"""
    return tf.multiply(input_activation, tf.nn.sigmoid(input_activation))

o_size_h = 168
o_size_w = 168
curiosity_enc_size = 64 #64 for int
vec_obs_size = 8
curiosity_strength=1.0
vis_obs_size = 1

# Action space: These subspaces are as follows: forward/backward/noop movement, 
# left/right/no-op movement, 
# clockwise/counterclockwise rotation of the camera/no-op, 
# and no-op/jump.
act_size = [3,3,3,2]
encoded_state = tf.placeholder(shape=[None, 128], dtype=tf.float32,name="encoded_state")
encoded_next_state = tf.placeholder(shape=[None, 128], dtype=tf.float32,name="encoded_next_state")
sel_action = tf.placeholder(shape=[None, 4], dtype=tf.float32,name="sel_action")


def create_inverse_model(encoded_state, encoded_next_state,selected_actions):
        """
        Creates inverse model TensorFlow ops for Curiosity module.
        Predicts action taken given current and future encoded states.
        :param encoded_state: Tensor corresponding to encoded current state.
        :param encoded_next_state: Tensor corresponding to encoded next state.
        """
        combined_input = tf.concat([encoded_state, encoded_next_state], axis=1)
        hidden = tf.layers.dense(combined_input, 256, activation=swish)
#         if self.brain.vector_action_space_type == "continuous":
#         pred_action = tf.layers.dense(hidden, act_size, activation=None)
#             squared_difference = tf.reduce_sum(tf.squared_difference(self.pred_action, self.selected_actions), axis=1)
#             self.inverse_loss = tf.reduce_mean(tf.dynamic_partition(squared_difference, self.mask, 2)[1])
#         else:
        pred_action = tf.concat(
                [tf.layers.dense(hidden, act_size[i], activation=tf.nn.softmax)
                 for i in range(len(act_size))], axis=1)
        
        cross_entropy = tf.reduce_sum(-tf.log(pred_action + 1e-10) * selected_actions, axis=1)
        inverse_loss = tf.reduce_mean(tf.dynamic_partition(cross_entropy, [0]*3999, 2)[1])
        
        return pred_action, inverse_loss

pred_actions = []
inverse_losses = []

pred_action, inverse_loss = create_inverse_model(encoded_state, encoded_next_state, sel_action)

pred_actions.append(pred_action)
inverse_losses.append(inverse_loss)
    
    
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)


# For int
pred_actions_Int = sess.run(pred_actions, feed_dict={encoded_state: encoded_state_int, 
                                                     encoded_next_state: encoded_nextstate_int,
                                                    sel_action: actions})
inverse_losses_Int = sess.run(inverse_losses, feed_dict={encoded_state: encoded_state_int, 
                                                     encoded_next_state: encoded_nextstate_int,
                                                    sel_action: actions})



ValueError: Dimensions must be equal, but are 11 and 4 for 'mul' (op: 'Mul') with input shapes: [?,11], [?,4].

In [40]:
pred_actions_Int[0][0]

array([0.26777884, 0.33520216, 0.397019  , 0.27890372, 0.3936167 ,
       0.32747957, 0.31436658, 0.3439591 , 0.3416743 , 0.5264839 ,
       0.47351614], dtype=float32)

In [34]:
"""FOR EXT + INT CASE"""


encoded_state_extint = np.load("F:\STUDY\MS Cognitive Science IITD\Internship\RTG Computational Cognition\Code\FastConceptMapping-main\data\\curiosity_encoding_ExtInt.npy")
encoded_nextstate_extint = np.load("F:\STUDY\MS Cognitive Science IITD\Internship\RTG Computational Cognition\Code\FastConceptMapping-main\data\\curiosity_encoding_nextstate_ExtInt.npy")
# encoded_state_extint = np.array(encoded_state_int,dtype=float)
# encoded_nextstate_extint = np.array(encoded_nextstate_int,dtype=float)
from numpy import argmax

tf.reset_default_graph()

def swish(input_activation):
    """Swish activation function. For more info: https://arxiv.org/abs/1710.05941"""
    return tf.multiply(input_activation, tf.nn.sigmoid(input_activation))

o_size_h = 168
o_size_w = 168
curiosity_enc_size = 128 #64 for int
vec_obs_size = 8
curiosity_strength=1.0
vis_obs_size = 1

# Action space: These subspaces are as follows: forward/backward/noop movement, 
# left/right/no-op movement, 
# clockwise/counterclockwise rotation of the camera/no-op, 
# and no-op/jump.
act_size = [3,3,3,2]

encoded_state = tf.placeholder(shape=[None, 256], dtype=tf.float32,name="encoded_state")
encoded_next_state = tf.placeholder(shape=[None, 256], dtype=tf.float32,name="encoded_next_state")


def create_inverse_model(encoded_state, encoded_next_state):
        """
        Creates inverse model TensorFlow ops for Curiosity module.
        Predicts action taken given current and future encoded states.
        :param encoded_state: Tensor corresponding to encoded current state.
        :param encoded_next_state: Tensor corresponding to encoded next state.
        """
        combined_input = tf.concat([encoded_state, encoded_next_state], axis=1)
        hidden = tf.layers.dense(combined_input, 256, activation=swish)
#         if self.brain.vector_action_space_type == "continuous":
#         pred_action = tf.layers.dense(hidden, act_size, activation=None)
#             squared_difference = tf.reduce_sum(tf.squared_difference(self.pred_action, self.selected_actions), axis=1)
#             self.inverse_loss = tf.reduce_mean(tf.dynamic_partition(squared_difference, self.mask, 2)[1])
#         else:
        pred_action = tf.concat(
                [tf.layers.dense(hidden, act_size[i], activation=tf.nn.softmax)
                 for i in range(len(act_size))], axis=1)
            
        return pred_action

#         cross_entropy = tf.reduce_sum(-tf.log(self.pred_action + 1e-10) * self.selected_actions, axis=1)
#         self.inverse_loss = tf.reduce_mean(tf.dynamic_partition(cross_entropy, self.mask, 2)[1])

pred_actions = []


pred_action = create_inverse_model(encoded_state, encoded_next_state)
    

pred_actions.append(pred_action)
    
    
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

# saver = tf.train.Saver()

# saver.restore(sess, ckpt_path_extint)
# saver.restore(sess, ckpt_path_int)


# For ext + int
pred_actions_ExtInt = sess.run(pred_actions, feed_dict={encoded_state: encoded_state_extint, encoded_next_state: encoded_nextstate_extint})



In [35]:
pred_actions_ExtInt = np.array(pred_actions_ExtInt).reshape(3999,11)

In [36]:
pred_action_argmax = [0]*3999
for index,act in enumerate(pred_actions_ExtInt):
    
        act_1 = argmax(act[0:3])
        act_2 = argmax(act[3:6])
        act_3 = argmax(act[6:9])
        act_4 = argmax(act[9:11])
        
        pred_action_argmax[index] = [act_1,act_2,act_3,act_4]

In [37]:
# np.array(pred_actions_ExtInt).shape
# argmax(pred_actions_ExtInt)
# pred_actions_ExtInt
pred_action_argmax[:]

[[1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2, 0],
 [1, 0, 2,

In [11]:
import matplotlib as plt
plt.hist(actions)

ModuleNotFoundError: No module named 'cycler'

In [18]:
logits = pred_actions_ExtInt[0][0]
probs = tf.nn.softmax(logits)
k = 3 # the first k=2 highest values
indices, values = tf.nn.top_k(probs, k=k)
print(indices,values)

Tensor("TopKV2_1:0", shape=(3,), dtype=float32) Tensor("TopKV2_1:1", shape=(3,), dtype=int32)


In [37]:
actions

array([[1, 1, 0, 1],
       [1, 1, 0, 1],
       [1, 1, 0, 1],
       ...,
       [1, 1, 0, 2],
       [1, 1, 0, 1],
       [1, 1, 0, 1]])

### Autoencoder

In [41]:
from tensorflow.python.keras import backend as K

from tensorflow.python.keras.models import Sequential,Model
from tensorflow.python.keras.layers import Conv3D, Conv2D, UpSampling2D,Conv2DTranspose
from tensorflow.python.keras.layers import ConvLSTM2D
from tensorflow.python.keras.layers import BatchNormalization
from tensorflow.python.keras.layers import Dense,MaxPooling2D,TimeDistributed,Input,concatenate,Flatten,Reshape,LSTM,Lambda

K.clear_session()

def swish(input_activation):
    """Swish activation function. For more info: https://arxiv.org/abs/1710.05941"""
    return tf.multiply(input_activation, tf.nn.sigmoid(input_activation))

inImg = Input(batch_shape=(None,168, 168, 3),name="input_1")
conv = Conv2D(filters=16, kernel_size=[8, 8], strides=[4, 4],activation=tf.nn.elu, name="conv_1")(inImg)
print(conv)
conv = Conv2D(filters=32, kernel_size=[4, 4], strides=[2, 2],activation=tf.nn.elu, name="conv_2")(conv)
print(conv)
flat = Reshape((19*19*32,))(conv)
print(flat)
dens = Dense(256,activation=swish,kernel_initializer=c_layers.variance_scaling_initializer(1.0), name="dens_1")(flat)
print(dens)
enc = Dense(256,activation=swish,kernel_initializer=c_layers.variance_scaling_initializer(1.0), name="dens_2")(dens)
print(enc)
de_dens = Dense(20*20*32,activation=swish,kernel_initializer=c_layers.variance_scaling_initializer(1.0), name="dens_3")(enc)
print(de_dens)
shaped = Reshape((20, 20, 32))(de_dens)
print(shaped)
de_conv = Conv2DTranspose(filters=16, kernel_size=[4, 4], strides=[2, 2],activation=tf.nn.elu, name="deconv_1")(shaped)
print(de_conv)

prediction = Conv2DTranspose(filters=3, kernel_size=[8, 8], strides=[4, 4],padding='same',activation=tf.nn.elu, name="deconv_3")(de_conv)
print(prediction)
model = Model(inputs=inImg, outputs=prediction)

model.compile(optimizer='adadelta',loss='mean_squared_error',metrics=['accuracy','mse'])
# adapt this path. Trained model is in the folder 'data'
model.load_weights('F:\STUDY\MS Cognitive Science IITD\Internship\RTG Computational Cognition\Code\zdh4d5ws2z-2\data\data\Autoencoder/aemodelAdam50E.h5')

intermediate_layer_model = Model(inputs=model.input,
                                 outputs=[model.get_layer('dens_2').output,model.get_layer('deconv_3').output])

Tensor("conv_1/Elu:0", shape=(?, 41, 41, 16), dtype=float32)
Tensor("conv_2/Elu:0", shape=(?, 19, 19, 32), dtype=float32)
Tensor("reshape/Reshape:0", shape=(?, 11552), dtype=float32)
Tensor("dens_1/Mul:0", shape=(?, 256), dtype=float32)
Tensor("dens_2/Mul:0", shape=(?, 256), dtype=float32)
Tensor("dens_3/Mul:0", shape=(?, 12800), dtype=float32)
Tensor("reshape_1/Reshape:0", shape=(?, 20, 20, 32), dtype=float32)
Tensor("deconv_1/Elu:0", shape=(?, 42, 42, 16), dtype=float32)
Tensor("deconv_3/Elu:0", shape=(?, 168, 168, 3), dtype=float32)


In [42]:
encAE = intermediate_layer_model.predict(obs)
encAE = encAE[0]

### Classifier

In [43]:
tf.reset_default_graph()

def swish(input_activation):
    """Swish activation function. For more info: https://arxiv.org/abs/1710.05941"""
    return tf.multiply(input_activation, tf.nn.sigmoid(input_activation))

o_size_h = 168
o_size_w = 168
num_layers = 2
h_size = 256
h_size_vec = 256
            
visual_in = tf.placeholder(shape=[None, o_size_h, o_size_w, 3], dtype=tf.float32,name="visual_observation_0")
labels = tf.placeholder(shape=[None,8], dtype=tf.int64,name="labels")

def create_vector_observation_encoder(observation_input, h_size, activation, num_layers, scope,reuse):
    with tf.variable_scope(scope):
        hidden_vec = observation_input
        for i in range(num_layers):
            hidden_vec = tf.layers.dense(hidden_vec, h_size, activation=activation, reuse=reuse,name="hidden_{}".format(i),kernel_initializer=c_layers.variance_scaling_initializer(1.0))
    return hidden_vec

def create_visual_observation_encoder(image_input, h_size, activation, num_layers, scope,reuse):
    with tf.variable_scope(scope):
        conv1 = tf.layers.conv2d(image_input, 16, kernel_size=[8, 8], strides=[4, 4],activation=tf.nn.elu, reuse=reuse, name="conv_1")
        conv2 = tf.layers.conv2d(conv1, 32, kernel_size=[4, 4], strides=[2, 2],activation=tf.nn.elu, reuse=reuse, name="conv_2")
        hidden_vis = c_layers.flatten(conv2)

    with tf.variable_scope(scope + '/' + 'flat_encoding'):
        hidden_flat = create_vector_observation_encoder(hidden_vis, h_size, activation,num_layers, scope, reuse)
    return hidden_flat

visual_encoders = []

encoded_visual = create_visual_observation_encoder(visual_in,h_size,swish,num_layers,"main_graph_0_encoder0", False)
visual_encoders.append(encoded_visual)
hidden = tf.concat(visual_encoders, axis=1)

out_acts = []
for o in range(8):
    out_acts.append(tf.layers.dense(hidden, 2, activation=tf.nn.softmax, use_bias=False,kernel_initializer=c_layers.variance_scaling_initializer(factor=0.01)))
print(out_acts)

output = tf.concat([tf.multinomial(tf.log(out_acts[k]), 1) for k in range(8)], axis=1)#sample outputs from log probdist
print(output)
#output = tf.round(out_act)
#normalized_logits = tf.identity(normalized_logits_flat, name='action')#has nan in places where prob is negative bc it it log(probs)

comparison = tf.equal(labels, output)

accuracy = tf.reduce_mean(tf.cast(comparison, dtype=tf.float32))

saver = tf.train.Saver()

[<tf.Tensor 'dense/Softmax:0' shape=(?, 2) dtype=float32>, <tf.Tensor 'dense_1/Softmax:0' shape=(?, 2) dtype=float32>, <tf.Tensor 'dense_2/Softmax:0' shape=(?, 2) dtype=float32>, <tf.Tensor 'dense_3/Softmax:0' shape=(?, 2) dtype=float32>, <tf.Tensor 'dense_4/Softmax:0' shape=(?, 2) dtype=float32>, <tf.Tensor 'dense_5/Softmax:0' shape=(?, 2) dtype=float32>, <tf.Tensor 'dense_6/Softmax:0' shape=(?, 2) dtype=float32>, <tf.Tensor 'dense_7/Softmax:0' shape=(?, 2) dtype=float32>]
Tensor("concat_1:0", shape=(?, 8), dtype=int64)


In [10]:
sess = tf.Session()
init = tf.global_variables_initializer()
saver.restore(sess, "F:\STUDY\MS Cognitive Science IITD\Internship\RTG Computational Cognition\Code\zdh4d5ws2z-2\data\data\Classifier\Model_lr0001_scaled/model.ckpt")
sess.run(init)
# adapt this path. Trained model is in the folder 'data'
outClass,actClass = sess.run([output,hidden], feed_dict = {visual_in: obs})

INFO:tensorflow:Restoring parameters from F:\STUDY\MS Cognitive Science IITD\Internship\RTG Computational Cognition\Code\zdh4d5ws2z-2\data\data\Classifier\Model_lr0001_scaled/model.ckpt


InvalidArgumentError: Unsuccessful TensorSliceReader constructor: Failed to get matching files on F:\STUDY\MS Cognitive Science IITD\Internship\RTG Computational Cognition\Code\zdh4d5ws2z-2\data\data\Classifier\Model_lr0001_scaled/model.ckpt: Not found: FindFirstFile failed for: F:/STUDY/MS Cognitive Science IITD/Internship/RTG Computational Cognition/Code/zdh4d5ws2z-2/data/data/Classifier/Model_lr0001_scaled : The system cannot find the path specified.
; No such process
	 [[Node: save/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save/Const_0_0, save/RestoreV2/tensor_names, save/RestoreV2/shape_and_slices)]]

Caused by op 'save/RestoreV2', defined at:
  File "c:\users\varad srivastava\anaconda3\envs\ml-agents\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\varad srivastava\anaconda3\envs\ml-agents\lib\runpy.py", line 85, in _run_code
   

In [ ]:
def getFlatLabel(label2D):
    flatLabels = []
    for label in label2D:
        flatLabel = np.zeros(8)
        if label[0] == 0:
            flatLabel[0] = 1
        elif label[0] == 1:
            flatLabel[1] = 1
        elif label[0] == 2:
            flatLabel[2] = 1
        elif label[0] == 3:
            flatLabel[3] = 1
        elif label[0] == 4:
            flatLabel[4] = 1
        if label[1] == 1:
            flatLabel[5] = 1
        if label[2] == 1:
            flatLabel[6] = 1
        if label[3] == 1:
            flatLabel[7] = 1
        flatLabels.append(flatLabel)
    return np.array(flatLabels)
flatL = getFlatLabel(y)[:,:-1]

In [ ]:
#calculte calssifier stats with same amount of positive as negative examples per concept 
#to do this negative examples are randomly sampled from all possible negative examples

accuracies, precisions, recalls, f1scores = [],[],[],[]
for c in range(7):
    accuracy, precision, recall, f1score = [],[],[],[]
    for i in range(1000):#repeat multiple times to avoid effect of random sampling for negative examples
        classIDs = np.where(flatL[:,c]==1)[0]
        numExp = np.shape(classIDs)[0]
        negExpIDs = np.where(flatL[:,c]==0)[0]
        np.random.shuffle(negExpIDs)
        negExpIDs = negExpIDs[:numExp]

        true_pos = np.sum(outClass[classIDs,c] == flatL[classIDs,c])
        true_neg = np.sum(outClass[negExpIDs,c] == flatL[negExpIDs,c])
        false_pos = np.sum(outClass[negExpIDs,c] != flatL[negExpIDs,c])
        false_neg = np.sum(outClass[classIDs,c] != flatL[classIDs,c])

        accuracy.append((true_neg+true_pos)/(numExp*2))
        precision.append(true_pos/(true_pos+false_pos))
        recall.append(true_pos/(true_pos+false_neg))
        f1score.append(2*(precision[i]*recall[i]/(precision[i]+recall[i])))
    accuracies.append(np.mean(accuracy)*100)
    precisions.append(np.mean(precision)*100)
    recalls.append(np.mean(recall)*100)
    f1scores.append(np.mean(f1score)*100)
classifier_stats = {'Accuracy': np.mean(accuracies),
            'Accuracies': np.array(accuracies),
            'Precisions': np.array(precisions),
            'Recalls': np.array(recalls),
            'F1Scores': np.array(f1scores)}
classifier_stats

## Save Activations

In [18]:
# np.save(balancedTestSetPath + 'encExt.npy',encExt)
# np.save(balancedTestSetPath + 'encInt.npy',encBal)
# np.save(balancedTestSetPath + 'encExtInt.npy',encExtInt)
# np.save(balancedTestSetPath + 'encAE.npy',encAE)
# np.save(balancedTestSetPath + 'encC.npy',actClass)
# np.save(balancedTestSetPath + 'outClass.npy',outClass)
np.save(balancedTestSetPath + 'labels.npy',y)
np.save(balancedTestSetPath + 'obs.npy',obs)
# adapt this path.
# figurePath = './Results/TowerTraining/Figures/AgentRewardComparisonsAdaTH/ActivationPatternsNormx2No-01-FlatC-50-50Test/'
# np.save(figurePath+'classifier_stats.npy',classifier_stats)